In [21]:
import os
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns

In [22]:
source_path = r"C:\Users\chench10\Downloads\-" 
file = 'MFile_cleaned_withLBname_11.28'

df_in = pd.read_json(os.path.join(source_path, file), compression='gzip', orient='records', lines=True)
df_copy = df_in.copy()

In [62]:
df = df_in[df_in['clusterNtables']>=8]
df.drop(columns=['clusterNtables', 'tableNclusters'], inplace=True)

C:\Users\chench10\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [65]:
tmp = pd.DataFrame(df.groupby(['table_id'])['cluster_id'].nunique()).reset_index()
tmp.columns = ['table_id','tableNclusters']
df = df.merge(tmp, on='table_id', how='left').sort_values(by='tableNclusters', ascending=False)

tmp = pd.DataFrame(df.groupby(['cluster_id'])['table_id'].nunique()).reset_index()
tmp.columns = ['cluster_id','clusterNtables']
df = df.merge(tmp, on='cluster_id', how='left')

In [66]:
print(df['table_id'].nunique())
print(df['cluster_id'].nunique())

1041
1330


In [69]:
df['cluster_id'] = df.groupby('cluster_id').ngroup()
df.sort_values('cluster_id').head()

,row_id,table_id,name,address,page_url,telephone,addressregion,streetaddress,addresslocality,addresscountry,longitude,latitude,telephone_,phone_object,E.164 format,telephoneNorm,cluster_id,tableNclusters,clusterNtables
14611,1206,LocalBusiness_australias.guide_September2020.j...,Hilton Cairns,"{'addresscountry': 'AU', 'streetaddress': '34 ...",https://www.australias.guide/qld/accommodation...,07 40502000,Queensland,34 ESPLANADE,CAIRNS,AU,145.7802036,-16.9229208,740502000,"{'country_code': 61, 'extension': None, 'natio...",61740502000,61740502000,0,3,8
11271,180,Hotel_kayak.com.au_September2020.json.gz,Hilton Cairns,"{'streetaddress': '34 Esplanade', 'addresscoun...",https://www.kayak.com.au/Cairns-Hotels.23637.h...,+61 740 502 000,Queensland,34 ESPLANADE,CAIRNS,AU,None,None,61740502000,"{'country_code': 61, 'extension': None, 'natio...",61740502000,61740502000,0,34,8
13183,803,Hotel_accommodationbroome.com_September2020.js...,Hilton Cairns Accommodation Broome,"{'streetaddress': '34 Esplanade, Cairns QLD 48...",https://www.accommodationbroome.com/accommodat...,07 4050 2000,QLD,"34 ESPLANADE, CAIRNS QLD 4870, AUSTRALIA",34 ESPLANADE,AU,None,None,740502000,"{'country_code': 61, 'extension': None, 'natio...",61740502000,61740502000,0,9,8
7417,3890,Hotel_hotelscombined.com_September2020.json.gz,Hilton Cairns,"{'streetaddress': '34 Esplanade', 'addresscoun...",https://www.hotelscombined.com/Hotel/Hilton_Ca...,+61 740 502 000,Queensland,34 ESPLANADE,CAIRNS,AU,None,None,61740502000,"{'country_code': 61, 'extension': None, 'natio...",61740502000,61740502000,0,79,8
13203,152,Hotel_accommodationairliebeach.com_September20...,Hilton Cairns Accommodation Airlie Beach,"{'streetaddress': '34 Esplanade, Cairns QLD 48...",https://www.accommodationairliebeach.com/accom...,07 4050 2000,QLD,"34 ESPLANADE, CAIRNS QLD 4870, AUSTRALIA",34 ESPLANADE,AU,None,None,740502000,"{'country_code': 61, 'extension': None, 'natio...",61740502000,61740502000,0,9,8


### Table Selection

In [20]:
# Based on some preliminary tests, my idea for table & entity selection is as follows:

# - start with top 30 tables with largets 'tableNclusters' as candidates; in each iteration, randomly select 500 entities 
#   calculate the number of cluster occurences for each table; save the results
# - redo the above step for N=10000 times, and calculate the mean of cluster occurences for each table
# - select the 15 tables with largest mean occurence. The underlying assumption is that, if a table is more likely to 
#   cover some randomly selected entities, it should have a higher probability of including the 500 selected entities
#   in the end.
# - do the above random sampling again on entities and select the 500 entities that are most likely to be included in the
#   15 tables selected from the last step.

In [23]:
# np.random.seed(42)
# table_idx = np.random.randint(0, ntables, 15)   # 15 is the number of selected tables
# table_candidates = tables[table_idx]

# cluster_idx = np.random.randint(0, ntables, 500)   # 500 is the number of selected clusters
# cluster_candidates = clusters[cluster_idx]

In [24]:
# initialize a dictionary to store cluster frequency
# cluster_fre = {key: 0 for key in range(ncluster)}

# for t in table_candidates:
#     clusters = df[df['table_id']==t]['cluster_id'].unique()
#     for c in clusters:
#         cluster_fre[c] += 1
        
# results =pd.DataFrame.from_dict(cluster_fre, orient='index', columns=['frequency']).sort_values('frequency', ascending=False)

In [25]:
# np.random.seed(42)

# sampling_tables = 50
# sampling_times = 100

# tables = np.array(df['table_id'].unique())[:sampling_tables]   
# clusters = np.array(df['cluster_id'].unique())

# array1 = np.ndarray((sampling_times, sampling_tables))
# for i in range(sampling_times):             # number of sampling attempts
#     table_candidates = tables
#     cluster_candidates = np.random.choice(clusters, 500, replace=False)   # 500 is the number of selected clusters

#     list_frequency =[]
#     for t in table_candidates:
#         l1 = set(df[df['table_id']==t].cluster_id.unique())  # clusters that are included in the table  
#         l2 = set(cluster_candidates)                         # choosen clusters

#         list_frequency.append(len(l1.intersection(l2)))      
    
#     #print(list_frequency)
#     #if mean(list_frequency) > 200:
#     #    break
#     array2 = np.append(array1, list_frequency).reshape(-1,sampling_tables)

In [26]:
# print(np.mean(array2, axis=0))

[1.13861386 1.02970297 0.77227723 0.67326733 0.65346535 0.3960396
 0.27722772 0.2970297  0.24752475 0.26732673 0.24752475 0.22772277
 0.16831683 0.18811881 0.20792079 0.20792079 0.16831683 0.11881188
 0.0990099  0.0990099  0.06930693 0.06930693 0.06930693 0.01980198
 0.10891089 0.11881188 0.10891089 0.0990099  0.02970297 0.08910891
 0.08910891 0.07920792 0.05940594 0.08910891 0.03960396 0.07920792
 0.0990099  0.02970297 0.04950495 0.05940594 0.01980198 0.03960396
 0.04950495 0.03960396 0.04950495 0.02970297 0.04950495 0.04950495
 0.05940594 0.01980198]


In [59]:
# idx = np.argpartition(np.mean(array2, axis=0), -30)[-30:]     # first narrow the tables to 30
# tables_selected = tables[idx]
# tables_selected

array(['Hotel_ihg.com_September2020.json.gz',
       'LocalBusiness_usharbors.com_September2020.json.gz',
       'Restaurant_ubereats.com_September2020.json.gz',
       'LocalBusiness_homify.com.co_September2020.json.gz',
       'LocalBusiness_acushops.com_September2020.json.gz',
       'LocalBusiness_saabshops.com_September2020.json.gz',
       'LocalBusiness_visitdenmark.com_September2020.json.gz',
       'LocalBusiness_homify.com.ar_September2020.json.gz',
       'Restaurant_opentable.com.au_September2020.json.gz',
       'LocalBusiness_infinitishops.com_September2020.json.gz',
       'Restaurant_theinfatuation.com_September2020.json.gz',
       'LocalBusiness_exoticautoshops.com_September2020.json.gz',
       'Hotel_marriott.co.uk_September2020.json.gz',
       'LocalBusiness_lexrepairshops.com_September2020.json.gz',
       'LocalBusiness_jagshops.com_September2020.json.gz',
       'LocalBusiness_homify.com.my_September2020.json.gz',
       'LocalBusiness_homify.com_September2020.

### Entity Selection

In [13]:
# select the 15 tables with largest mean occurence (15 is the number of selected tables)
# then do the above random sampling again on entities and select the 500 entity with largest mean inclusion

In [28]:
# # initialize a dictionary to store cluster frequency
# cluster_fre = {key: 0 for key in range(1, ncluster+1)}

# for t in tables_selected:
#     clusters = df[df['table_id']==t]['cluster_id'].unique()
#     for c in clusters:
#         cluster_fre[c] += 1

In [1]:
# cluster_fre
# results =pd.DataFrame.from_dict(cluster_fre, orient='index', columns=['frequency']).sort_values('frequency', ascending=False)

# clusters_selected = results[:500].index
# clusters_selected

In [34]:
# tmp = df.set_index('cluster_id').loc[clusters_selected]

In [2]:
# tables_selected2 = tables_selected[[4,5,9,11,13,14, 17, 18,20,21,22,23, 24, 25]]
# tables_selected2

In [64]:
# tmp['1']=tmp.reset_index().apply(lambda x: x['table_id'] in tables_selected2, axis=1)
# tmp2=tmp[tmp['1']==True].sort_values('clusterNtables')

### Train/Test splitng

In [113]:
nclusters = df['cluster_id'].nunique()
tables = df['table_id'].unique()
ntables = df['table_id'].nunique()

In [71]:
# create a dictionary where the keys are table_id and values are lists of cluster_id
results = {}
for t in tables:
    results[t] = []

In [72]:
for c in range(nclusters):
    tables_c = df[df['cluster_id']==c]['table_id'].unique()
    for t in tables_c:
        results[t].append(c)

In [101]:
results2 = {}
for t in tables:
    results2[t] = np.zeros(nclusters)
    idxes = results[t]
    results2[t][idxes] = 1

In [106]:
overview = pd.DataFrame.from_dict(results2).to_numpy()
overview  # axis-1: table_ids (ordered as in the series tables); axis-0: cluster_ids 

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [117]:
# there are 1041 unique tables in total and assign the tables to 3 groups (3:2:3)
# number of tables in the train set: 391
# number of tables in the validation set: 260
# number of tables in the test set: 390

ntrain_tables = 391
nval_tables = 260
ntest_tables =391

In [118]:
# target arrays
ttrain = np.sum(overview, axis=1) * 0.375
tval = np.sum(overview, axis=1) * 0.25
ttest = np.sum(overview, axis=1) * 0.375

In [252]:
np.random.seed(42)

# initialize array/list to store results
N = 10000
loss_scores = np.zeros(N)
splits = []

for n in range(N):  
    # 1) randomly select tables to form train/val/test
    # sampling for train
    candidates0 = np.arange(ntables)
    train_tables = np.random.choice(candidates0, ntrain_tables, replace=False)
    # sampling for val 
    candidates1 = np.delete(candidates0, train_tables)
    val_tables = np.random.choice(candidates1, nval_tables, replace=False)
    # (sampling) for test
    test_tables =  np.delete(candidates0, np.append(train_tables,val_tables))

    # 2) compute the realized target array
    rtrain = np.sum(overview[:, train_tables], axis=1)
    rval = np.sum(overview[:, val_tables], axis=1)
    rtest = np.sum(overview[:, test_tables], axis=1)
    
    # try best to ensure that each cluster has at least one representative in each group, 
    # penalty if one cluster is not included
    train_zeros = len(np.where(rtrain==0)[0])
    val_zeros = len(np.where(rval==0)[0])
    test_zeros = len(np.where(rtest==0)[0])
           # penalty for deviation from the target array
    loss = np.linalg.norm(rtrain - ttrain) + np.linalg.norm(rval - tval) + np.linalg.norm(rtest - ttest) + \
           train_zeros + val_zeros + test_zeros
    
    # save the results
    loss_scores[n] = loss
    splits.append((train_tables, val_tables, test_tables))
               
    # 3) repeat the above procedure N times, and select the splitting strategy with least loss
    n += 1

In [253]:
# i-th iteration with minimum loss
idx_minloss = np.where(loss_scores==np.min(loss_scores))[0][0]
print(idx_minloss)
print(loss_scores[idx_minloss])

# splitting strategy
splitting_minloss = splits[idx_minloss]
selected_train_tables = tables[splitting_minloss[0]]
selected_val_tables = tables[splitting_minloss[1]]
selected_test_tables = tables[splitting_minloss[2]]

2936
157.94863519320583


In [254]:
print('training set:')
print(tables[splitting_minloss[0]])

training set:
['Restaurant_yellowpages.com_September2020.json.gz'
 'Hotel_dayuse.ie_September2020.json.gz'
 'LocalBusiness_twincitiesemrconsulting.com_September2020.json.gz'
 'LocalBusiness_hallofdistinction.com_September2020.json.gz'
 'LocalBusiness_tuich.site_September2020.json.gz'
 'LocalBusiness_rci.edu_September2020.json.gz'
 'Restaurant_miamiandbeaches.com_September2020.json.gz'
 'Hotel_hotelscombined.in_September2020.json.gz'
 'Restaurant_slicelife.com_September2020.json.gz'
 'Hotel_pinkuk.com_September2020.json.gz'
 'LocalBusiness_poolrepaircompaniesnearme.com_September2020.json.gz'
 'LocalBusiness_platinumluxuryfleet.com_September2020.json.gz'
 'LocalBusiness_purposehealingcenter.com_September2020.json.gz'
 'LocalBusiness_webcongress.co_September2020.json.gz'
 'LocalBusiness_alcoholrehabcenter.co_September2020.json.gz'
 'LocalBusiness_bestnewscenter.com_September2020.json.gz'
 'Restaurant_squaremeal.co.uk_September2020.json.gz'
 'LocalBusiness_concretorsnearme.com.au_September

In [255]:
print('validation set:')
print(tables[splitting_minloss[1]])

validation set:
['LocalBusiness_localcarpetcleanersnearme.com_September2020.json.gz'
 'LocalBusiness_visitdenmark.fr_September2020.json.gz'
 'LocalBusiness_neckandback.net_September2020.json.gz'
 'LocalBusiness_visitodense.com_September2020.json.gz'
 'Restaurant_dinnerbooking.com_September2020.json.gz'
 'Restaurant_opentable.jp_September2020.json.gz'
 'Hotel_kayak.dk_September2020.json.gz'
 'Hotel_holidayinn.com_September2020.json.gz'
 'LocalBusiness_jagshops.com_September2020.json.gz'
 'Restaurant_opentable.com.mx_September2020.json.gz'
 'LocalBusiness_bestengineeredwoodflooring.com_September2020.json.gz'
 'LocalBusiness_listingdirectori.org_September2020.json.gz'
 'LocalBusiness_electriciansnearme.net.au_September2020.json.gz'
 'LocalBusiness_insurancebrokersnearme.com.au_September2020.json.gz'
 'LocalBusiness_bestratedwindows.com_September2020.json.gz'
 'Hotel_momondo.co.uk_September2020.json.gz'
 'LocalBusiness_daffyducts.com_September2020.json.gz'
 'LocalBusiness_trucksalesnearme.

In [256]:
print('test set:')
print(tables[splitting_minloss[2]])

test set:
['LocalBusiness_benzshops.com_September2020.json.gz'
 'LocalBusiness_minirepairshops.com_September2020.json.gz'
 'LocalBusiness_lrshops.com_September2020.json.gz'
 'LocalBusiness_saabshops.com_September2020.json.gz'
 'LocalBusiness_acushops.com_September2020.json.gz'
 'LocalBusiness_infinitishops.com_September2020.json.gz'
 'LocalBusiness_homify.sg_September2020.json.gz'
 'LocalBusiness_homify.co.uk_September2020.json.gz'
 'LocalBusiness_homify.es_September2020.json.gz'
 'LocalBusiness_homify.com.mx_September2020.json.gz'
 'LocalBusiness_bigdirectori.com_September2020.json.gz'
 'LocalBusiness_homify.com_September2020.json.gz'
 'LocalBusiness_checkthemout.biz_September2020.json.gz'
 'LocalBusiness_homify.ph_September2020.json.gz'
 'LocalBusiness_ibiznet.co_September2020.json.gz'
 'LocalBusiness_homify.cl_September2020.json.gz'
 'LocalBusiness_1pointweb.net_September2020.json.gz'
 'Hotel_marriott.com_September2020.json.gz'
 'Hotel_kayak.com_September2020.json.gz'
 'LocalBusines